In [ ]:
from pandas import DataFrame, concat
import json

In [ ]:
from lyricsgenius import Genius
token = 'utWUfzSpqx5D-0h6kdlftC_wWX9s2iPsMP5T5QP0phbGprepCLAjRBZP3cmnhZhB'
genius = Genius(token, sleep_time=1)

In [ ]:
from utils import ArtistSongs, Artists, Charts, Songs, Lyrics, LyricSentiment
Artists.genius = genius
ArtistSongs.genius = genius
Charts.genius = genius
Songs.genius = genius
Lyrics.genius = genius

In [ ]:
Charts.chart_download()
ArtistSongs.load_from_charts()
Lyrics.load_from_artist_songs()

In [ ]:
def id(song) -> int:
    return song._id

def name(song) -> str:
    return song.title

def artist(song) -> str:
    return song.artist

def artist_id(song) -> int:
    song_json = song.to_json()
    song_json = json.loads(song_json)
    return song_json.get('primary_artist', {}).get('id', None)

def language(song) -> str:
    song_json = song.to_json()
    song_json = json.loads(song_json)
    return song_json.get('language', None)

def lyrics(song) -> str:
    lyrics = Lyrics.lyrics(song._id)
    lyrics = lyrics.split('\n')[1:]
    return " ".join(lyrics)

def image(song) -> str:
    song_json = song.to_json()
    song_json = json.loads(song_json)
    return song_json.get('song_art_image_url', None)

def url(song) -> str:
    song_json = song.to_json()
    song_json = json.loads(song_json)
    return song_json.get('url', None)

def map_song(song) -> dict:
    return {
        'ID': id(song),
        'Name': name(song),
        'Artist': artist(song),
        'Artist ID': artist_id(song),
        'Language': language(song),
        'Lyrics': lyrics(song),
        'Image': image(song),
        'URL': url(song)
    }


songs = Songs.songs()
songs_dict = list(map(map_song, songs))
songs_df = DataFrame(songs_dict)
songs_df.to_csv('songs.csv')

In [ ]:
lyrics = songs_df['Lyrics']
sentiments = lyrics.apply(LyricSentiment.sentiments).to_list()
sentiments_df = DataFrame(sentiments)
final_df = concat([songs_df['ID'], sentiments_df], axis=1)
final_df.to_csv('sentiments.csv')